In [16]:
import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer

from sklearn.model_selection import StratifiedKFold, cross_val_score,train_test_split

from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb
from xgboost import XGBClassifier
%matplotlib inline 

In [322]:
train = pd.read_csv("train.csv", sep = '\t', index_col = 0)
test = pd.read_csv("test.csv", sep = '\t', index_col = 0)
train.shape, test.shape

((30500, 346), (4166, 346))

In [120]:
for column in train:
    print(column, train[column].unique().size, train[column].size)

0 2 30500
1 2 30500
2 2 30500
3 2 30500
4 2 30500
5 2 30500
6 2 30500
7 13 30500
8 2 30500
9 2 30500
10 2 30500
11 2 30500
12 2 30500
13 859 30500
14 2 30500
15 2 30500
16 2 30500
17 2 30500
18 2 30500
19 2 30500
20 2 30500
21 2 30500
22 2 30500
23 2 30500
24 2 30500
25 2 30500
26 2 30500
27 2 30500
28 2 30500
29 2 30500
30 2 30500
31 2 30500
32 2 30500
33 2 30500
34 2 30500
35 2 30500
36 2 30500
37 2 30500
38 2 30500
39 2 30500
40 2 30500
41 2 30500
42 2 30500
43 2 30500
44 2 30500
45 2 30500
46 2 30500
47 2 30500
48 2 30500
49 2 30500
50 2 30500
51 2 30500
52 2 30500
53 2 30500
54 2 30500
55 2 30500
56 2 30500
57 2 30500
58 2 30500
59 2 30500
60 2 30500
61 2 30500
62 2 30500
63 2 30500
64 2 30500
65 2 30500
66 573 30500
67 1314 30500
68 668 30500
69 395 30500
70 803 30500
71 1388 30500
72 1241 30500
73 1462 30500
74 1187 30500
75 833 30500
76 680 30500
77 1020 30500
78 481 30500
79 236 30500
80 385 30500
81 484 30500
82 872 30500
83 1228 30500
84 404 30500
85 1099 30500
86 1996 30500

In [66]:
#Validation strategy
skf = StratifiedKFold(n_splits = 5, shuffle = True)

/home/anton/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [323]:
useless_columns = []
for column in test:
    if(test[column].unique().size == 1 and column != '0'):
        useless_columns.append((column, test[column].unique()[0]))
#useless_columns

In [324]:
dropped_train = train.copy()
dropped_test = test.copy()
for column in useless_columns:
    dropped_train = dropped_train[dropped_train[column[0]] == column[1]]
    dropped_train = dropped_train.drop([column[0]], axis = 1)
    dropped_test = dropped_test[dropped_test[column[0]] == column[1]]
    dropped_test = dropped_test.drop([column[0]], axis = 1)
dropped_train = dropped_train.reset_index(drop=True)
dropped_test = dropped_test.reset_index(drop=True)

In [325]:
X = dropped_train.drop(['0'], axis = 1)
test_X = dropped_test.drop(['0'], axis = 1)
y = dropped_train['0']

In [61]:
xgb_train = xgb.DMatrix(X, y, feature_names=X.columns)
parameters = {
    #default
    'objective': 'reg:logistic',
    'eta': 0.01, #reduced to get higher score
    'silent': 1,
    "nthread": 4,
    "random_seed": 1,
    "eval_metric": 'auc',
    
    # regularization parameters
    'max_leaves': 32,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    
    #lightgbm approach
    'tree_method': 'hist',
    'grow_policy': 'lossguide'
}
num_rounds = 1000

In [62]:
results = xgb.cv(parameters, xgb_train, num_rounds, early_stopping_rounds=10,
                 folds=skf, verbose_eval=1)

[0]	train-auc:0.686921+0.00711464	test-auc:0.669792+0.00911324
[1]	train-auc:0.71261+0.00440126	test-auc:0.691024+0.00208164
[2]	train-auc:0.719616+0.00528533	test-auc:0.697264+0.00142582
[3]	train-auc:0.720747+0.0038485	test-auc:0.697935+0.00164156
[4]	train-auc:0.724554+0.00450421	test-auc:0.700288+0.00163568
[5]	train-auc:0.725503+0.00458379	test-auc:0.700615+0.00150066
[6]	train-auc:0.726429+0.00334637	test-auc:0.700112+0.00305091
[7]	train-auc:0.727618+0.00370549	test-auc:0.701634+0.00306945
[8]	train-auc:0.728812+0.00334239	test-auc:0.703044+0.00282758
[9]	train-auc:0.729455+0.00308532	test-auc:0.703257+0.0034885
[10]	train-auc:0.730061+0.00310079	test-auc:0.703011+0.00341908
[11]	train-auc:0.730851+0.00437407	test-auc:0.704048+0.00200873
[12]	train-auc:0.730988+0.00357005	test-auc:0.704084+0.00195672
[13]	train-auc:0.731775+0.00303332	test-auc:0.704857+0.00295206
[14]	train-auc:0.732313+0.00265434	test-auc:0.704948+0.00342137
[15]	train-auc:0.732755+0.00269204	test-auc:0.705341+

[129]	train-auc:0.76063+0.00330422	test-auc:0.718978+0.00421702
[130]	train-auc:0.760853+0.00330916	test-auc:0.719154+0.00427056
[131]	train-auc:0.761118+0.00323771	test-auc:0.719234+0.00433576
[132]	train-auc:0.761434+0.0033063	test-auc:0.719307+0.00433977
[133]	train-auc:0.761612+0.00328675	test-auc:0.719326+0.00436969
[134]	train-auc:0.761946+0.00322756	test-auc:0.719462+0.00441348
[135]	train-auc:0.762213+0.00331176	test-auc:0.719643+0.00439142
[136]	train-auc:0.76248+0.00330506	test-auc:0.71979+0.00433244
[137]	train-auc:0.762739+0.00318664	test-auc:0.719841+0.00438277
[138]	train-auc:0.762943+0.00316986	test-auc:0.719933+0.00438391
[139]	train-auc:0.763139+0.00316295	test-auc:0.720013+0.00433638
[140]	train-auc:0.763305+0.00309577	test-auc:0.720046+0.00436964
[141]	train-auc:0.763489+0.00304484	test-auc:0.720111+0.00436773
[142]	train-auc:0.763691+0.00305505	test-auc:0.720142+0.0043611
[143]	train-auc:0.763827+0.00303815	test-auc:0.720199+0.00435465
[144]	train-auc:0.764079+0.003

KeyboardInterrupt: 

In [35]:
xclas = XGBClassifier(
    n_estimators = 250,
    early_stoppeng_rounds = 10,
    objective = 'reg:logistic',
    eta = 0.05, #reduced to get higher score
    silent = 1,
    nthread = -1,
    random_seed = 1,
    eval_metric = 'auc',
    
    # regularization parameters
    max_leaves = 32,
    subsample = 0.8,
    colsample_bytree = 0.8,
    
    #lightgbm approach
    tree_method = 'hist',
    grow_policy = 'lossguide'
)  # and for classifier  


In [37]:
xclas.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, early_stoppeng_rounds=10, eta=0.05,
       eval_metric='auc', gamma=0, grow_policy='lossguide',
       learning_rate=0.1, max_delta_step=0, max_depth=3, max_leaves=32,
       min_child_weight=1, missing=None, n_estimators=250, n_jobs=1,
       nthread=-1, objective='reg:logistic', random_seed=1, random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None, silent=1,
       subsample=0.8, tree_method='hist')

In [75]:
dropped_test['0'] = xclas.predict_proba(test_X)[:,1]
dropped_test['0'].to_csv('submissionXGB.csv',header = True, index_label = True)

In [330]:
import lightgbm as lgb
n_rounds = 10000


categ_columns = list(filter(lambda x: X[x].unique().size <= 100, X.select_dtypes(include=['int'])))
categ_columns

['1',
 '2',
 '3',
 '4',
 '6',
 '8',
 '9',
 '10',
 '12',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '95',
 '96',
 '97',
 '128',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '139',
 '142',
 '143',
 '144',
 '145',
 '146',
 '147',
 '148',
 '151',
 '152',
 '153',
 '154',
 '155',
 '156',
 '157',
 '158',
 '159',
 '160',
 '161',
 '162',
 '163',
 '165',
 '166',
 '167',
 '168',
 '169',
 '170',
 '171',
 '172',
 '173',
 '174',
 '175',
 '176',
 '177',
 '179',
 '180',
 '181',
 '182',
 '183',
 '184',
 '185',
 '187',
 '189',
 '190',
 '191',
 '215',
 '216',
 '217',
 '218',
 '219',
 '220',
 '221',
 '222',
 '223',
 '224',
 '225',
 '226',
 '227',
 '228',
 '229',
 '230',
 '231',
 '232',

In [125]:
lgb_train = lgb.Dataset(X, label=y, free_raw_data=False, categorical_feature=categ_columns)

In [126]:
parameters = {
    #
    'objective': 'binary',
    'learning_rate': 0.01,
    'num_threads': -1,
    "metric": 'auc',
    'num_leaves': 63,
    
    #regularization
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'subsample_freq': 1,
    'min_data_in_leaf': 15,
    'cat_smooth': 10,
    'min_data_per_group': 100
}
result = lgb.cv(parameters, lgb_train, n_rounds, folds=skf.split(X, y), early_stopping_rounds=400, verbose_eval=2)

/home/anton/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[2]	cv_agg's auc: 0.696743 + 0.00746883
[4]	cv_agg's auc: 0.705989 + 0.00736365
[6]	cv_agg's auc: 0.711842 + 0.00665952
[8]	cv_agg's auc: 0.715702 + 0.00681726
[10]	cv_agg's auc: 0.716684 + 0.00709507
[12]	cv_agg's auc: 0.717535 + 0.00701375
[14]	cv_agg's auc: 0.718872 + 0.00695028
[16]	cv_agg's auc: 0.719125 + 0.00679141
[18]	cv_agg's auc: 0.720052 + 0.00686615
[20]	cv_agg's auc: 0.720899 + 0.00681132
[22]	cv_agg's auc: 0.720943 + 0.00668422
[24]	cv_agg's auc: 0.721136 + 0.00668417
[26]	cv_agg's auc: 0.721018 + 0.00717854
[28]	cv_agg's auc: 0.721214 + 0.00752637
[30]	cv_agg's auc: 0.721613 + 0.00730784
[32]	cv_agg's auc: 0.721738 + 0.00768514
[34]	cv_agg's auc: 0.722243 + 0.00776007
[36]	cv_agg's auc: 0.722385 + 0.00772694
[38]	cv_agg's auc: 0.722312 + 0.00783191
[40]	cv_agg's auc: 0.722704 + 0.00770514
[42]	cv_agg's auc: 0.722768 + 0.00779468
[44]	cv_agg's auc: 0.722941 + 0.00742977
[46]	cv_agg's auc: 0.722901 + 0.0074737
[48]	cv_agg's auc: 0.723043 + 0.00770746
[50]	cv_agg's auc: 0.

[396]	cv_agg's auc: 0.743293 + 0.00595026
[398]	cv_agg's auc: 0.743319 + 0.00595371
[400]	cv_agg's auc: 0.743383 + 0.0059406
[402]	cv_agg's auc: 0.743401 + 0.00597655
[404]	cv_agg's auc: 0.743392 + 0.00597805
[406]	cv_agg's auc: 0.743467 + 0.00593767
[408]	cv_agg's auc: 0.743462 + 0.00591986
[410]	cv_agg's auc: 0.743484 + 0.00594347
[412]	cv_agg's auc: 0.74355 + 0.00593278
[414]	cv_agg's auc: 0.743576 + 0.00590347
[416]	cv_agg's auc: 0.743617 + 0.00595164
[418]	cv_agg's auc: 0.743634 + 0.00593297
[420]	cv_agg's auc: 0.743708 + 0.00599707
[422]	cv_agg's auc: 0.743745 + 0.00596275
[424]	cv_agg's auc: 0.743717 + 0.00594261
[426]	cv_agg's auc: 0.743731 + 0.00591933
[428]	cv_agg's auc: 0.74372 + 0.00592828
[430]	cv_agg's auc: 0.743774 + 0.00591554
[432]	cv_agg's auc: 0.743832 + 0.00587289
[434]	cv_agg's auc: 0.743892 + 0.00587588
[436]	cv_agg's auc: 0.743925 + 0.00586224
[438]	cv_agg's auc: 0.744011 + 0.0058987
[440]	cv_agg's auc: 0.744005 + 0.00591897
[442]	cv_agg's auc: 0.744023 + 0.00592

[790]	cv_agg's auc: 0.746089 + 0.00549834
[792]	cv_agg's auc: 0.746065 + 0.00545164
[794]	cv_agg's auc: 0.746077 + 0.00545858
[796]	cv_agg's auc: 0.746048 + 0.00543135
[798]	cv_agg's auc: 0.746019 + 0.00541797
[800]	cv_agg's auc: 0.746009 + 0.00541359
[802]	cv_agg's auc: 0.74604 + 0.0054254
[804]	cv_agg's auc: 0.746057 + 0.0053926
[806]	cv_agg's auc: 0.746025 + 0.00535412
[808]	cv_agg's auc: 0.745996 + 0.00535118
[810]	cv_agg's auc: 0.745954 + 0.00532969
[812]	cv_agg's auc: 0.745965 + 0.00536756
[814]	cv_agg's auc: 0.745973 + 0.0052838
[816]	cv_agg's auc: 0.745981 + 0.00527347
[818]	cv_agg's auc: 0.745975 + 0.0052412
[820]	cv_agg's auc: 0.745975 + 0.00526379
[822]	cv_agg's auc: 0.745993 + 0.005231
[824]	cv_agg's auc: 0.745991 + 0.00519333
[826]	cv_agg's auc: 0.745976 + 0.00522732
[828]	cv_agg's auc: 0.745974 + 0.00525889
[830]	cv_agg's auc: 0.74601 + 0.0052076
[832]	cv_agg's auc: 0.745982 + 0.00520271
[834]	cv_agg's auc: 0.746017 + 0.00519647
[836]	cv_agg's auc: 0.745989 + 0.00519322
[

KeyboardInterrupt: 

In [340]:
lgbm = lgb.train(parameters, lgb_train, 730)

/home/anton/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


In [128]:
dropped_test['0'] = lgbm.predict(test_X)
dropped_test['0'].to_csv('submissionLGBMmain.csv',header = True, index_label = True)

In [84]:
dropped_test['0']

0       0.105377
1       0.465623
2       0.357635
3       0.320813
4       0.730954
5       0.352463
6       0.072515
7       0.138354
8       0.108674
9       0.348798
10      0.106325
11      0.567821
12      0.039514
13      0.089150
14      0.739152
15      0.207213
16      0.067546
17      0.243192
18      0.261968
19      0.088707
20      0.049127
21      0.231063
22      0.143130
23      0.048334
24      0.552414
25      0.257327
26      0.438877
27      0.097641
28      0.057047
29      0.165648
          ...   
4136    0.111838
4137    0.042690
4138    0.067130
4139    0.116391
4140    0.090692
4141    0.198158
4142    0.070217
4143    0.088947
4144    0.073348
4145    0.138743
4146    0.048832
4147    0.126818
4148    0.089450
4149    0.211724
4150    0.156682
4151    0.086837
4152    0.127043
4153    0.338519
4154    0.250059
4155    0.202748
4156    0.488818
4157    0.133157
4158    0.167840
4159    0.288266
4160    0.036185
4161    0.072479
4162    0.096408
4163    0.0767

In [85]:
import catboost as ctb
parameters = {
    'loss_function':'Logloss',
    'eval_metric': 'AUC',
    'iterations': 650,
    'learning_rate': 0.03,
    'random_seed': 42,
    'use_best_model': False,
    'od_wait': 1000,
    'od_type': 'Iter'
}
for train, val in skf.split(X,y):
    temp_ctb_train = ctb.Pool(X.loc[train], y.loc[train])
    temp_ctb_val = ctb.Pool(X.loc[val], y.loc[val])
    temp_model = ctb.train(temp_ctb_train, parameters, logging_level='Verbose', eval_set=temp_ctb_val)

In [343]:
for train, val in skf.split(X,y):
    ctb_model = ctb.CatBoostClassifier(iterations=1500, learning_rate=0.03, depth=6, eval_metric='AUC', 
                                       random_seed=42, rsm=0.8, l2_leaf_reg=2, od_wait=50, od_type='Iter')
    ctb_model.fit(X.loc[train], y.loc[train], np.where(X.dtypes != np.float)[0],use_best_model=True, eval_set=(X.loc[val], y.loc[val]), verbose=True)

0:	learn: 0.6357840	test: 0.6220235	best: 0.6220235 (0)	total: 115ms	remaining: 2m 53s
1:	learn: 0.6625132	test: 0.6458576	best: 0.6458576 (1)	total: 215ms	remaining: 2m 41s
2:	learn: 0.6650261	test: 0.6490779	best: 0.6490779 (2)	total: 343ms	remaining: 2m 51s
3:	learn: 0.6730356	test: 0.6548502	best: 0.6548502 (3)	total: 499ms	remaining: 3m 6s
4:	learn: 0.6716097	test: 0.6529658	best: 0.6548502 (3)	total: 643ms	remaining: 3m 12s
5:	learn: 0.6700749	test: 0.6527321	best: 0.6548502 (3)	total: 757ms	remaining: 3m 8s
6:	learn: 0.6760298	test: 0.6608585	best: 0.6608585 (6)	total: 874ms	remaining: 3m 6s
7:	learn: 0.6794855	test: 0.6646273	best: 0.6646273 (7)	total: 1.01s	remaining: 3m 7s
8:	learn: 0.6821542	test: 0.6680107	best: 0.6680107 (8)	total: 1.14s	remaining: 3m 9s
9:	learn: 0.6843039	test: 0.6696514	best: 0.6696514 (9)	total: 1.26s	remaining: 3m 7s
10:	learn: 0.6844638	test: 0.6689766	best: 0.6696514 (9)	total: 1.46s	remaining: 3m 17s
11:	learn: 0.6847764	test: 0.6692054	best: 0.669

KeyboardInterrupt: 

In [349]:
ctb_model = ctb.CatBoostClassifier(iterations=800, learning_rate=0.03, depth=6, eval_metric='AUC', 
                                       random_seed=42, rsm=0.8, l2_leaf_reg=2, od_wait=50, od_type='Iter')
ctb_model.fit(X, y, np.where(X.dtypes != np.float)[0], verbose=True)

0:	learn: 0.5995459	total: 146ms	remaining: 1m 56s
1:	learn: 0.6770067	total: 281ms	remaining: 1m 52s
2:	learn: 0.6815969	total: 436ms	remaining: 1m 55s
3:	learn: 0.6782260	total: 575ms	remaining: 1m 54s
4:	learn: 0.6794455	total: 731ms	remaining: 1m 56s
5:	learn: 0.6800332	total: 868ms	remaining: 1m 54s
6:	learn: 0.6797430	total: 1.02s	remaining: 1m 56s
7:	learn: 0.6814659	total: 1.16s	remaining: 1m 54s
8:	learn: 0.6811045	total: 1.3s	remaining: 1m 54s
9:	learn: 0.6800273	total: 1.43s	remaining: 1m 52s
10:	learn: 0.6804459	total: 1.57s	remaining: 1m 52s
11:	learn: 0.6840521	total: 1.71s	remaining: 1m 52s
12:	learn: 0.6848669	total: 1.86s	remaining: 1m 52s
13:	learn: 0.6839954	total: 2s	remaining: 1m 52s
14:	learn: 0.6834102	total: 2.15s	remaining: 1m 52s
15:	learn: 0.6833561	total: 2.3s	remaining: 1m 52s
16:	learn: 0.6848977	total: 2.46s	remaining: 1m 53s
17:	learn: 0.6852425	total: 2.59s	remaining: 1m 52s
18:	learn: 0.6868873	total: 2.75s	remaining: 1m 52s
19:	learn: 0.6878825	total:

158:	learn: 0.7458644	total: 30s	remaining: 2m
159:	learn: 0.7460138	total: 30.1s	remaining: 2m
160:	learn: 0.7462082	total: 30.3s	remaining: 2m
161:	learn: 0.7464287	total: 30.5s	remaining: 2m
162:	learn: 0.7465819	total: 30.6s	remaining: 1m 59s
163:	learn: 0.7468582	total: 30.8s	remaining: 1m 59s
164:	learn: 0.7470320	total: 31s	remaining: 1m 59s
165:	learn: 0.7473454	total: 31.1s	remaining: 1m 58s
166:	learn: 0.7474813	total: 31.3s	remaining: 1m 58s
167:	learn: 0.7476338	total: 31.4s	remaining: 1m 58s
168:	learn: 0.7477525	total: 31.6s	remaining: 1m 57s
169:	learn: 0.7478708	total: 31.7s	remaining: 1m 57s
170:	learn: 0.7479851	total: 31.9s	remaining: 1m 57s
171:	learn: 0.7481751	total: 32s	remaining: 1m 56s
172:	learn: 0.7482732	total: 32.2s	remaining: 1m 56s
173:	learn: 0.7485352	total: 32.3s	remaining: 1m 56s
174:	learn: 0.7487628	total: 32.4s	remaining: 1m 55s
175:	learn: 0.7488843	total: 32.6s	remaining: 1m 55s
176:	learn: 0.7490884	total: 32.7s	remaining: 1m 55s
177:	learn: 0.7

314:	learn: 0.7693267	total: 58.2s	remaining: 1m 29s
315:	learn: 0.7693882	total: 58.4s	remaining: 1m 29s
316:	learn: 0.7695339	total: 58.6s	remaining: 1m 29s
317:	learn: 0.7696301	total: 58.7s	remaining: 1m 29s
318:	learn: 0.7698582	total: 58.9s	remaining: 1m 28s
319:	learn: 0.7699669	total: 59.1s	remaining: 1m 28s
320:	learn: 0.7700711	total: 59.2s	remaining: 1m 28s
321:	learn: 0.7701750	total: 59.7s	remaining: 1m 28s
322:	learn: 0.7702889	total: 1m	remaining: 1m 28s
323:	learn: 0.7703961	total: 1m	remaining: 1m 28s
324:	learn: 0.7705623	total: 1m	remaining: 1m 28s
325:	learn: 0.7706939	total: 1m 1s	remaining: 1m 28s
326:	learn: 0.7708831	total: 1m 1s	remaining: 1m 28s
327:	learn: 0.7710264	total: 1m 1s	remaining: 1m 28s
328:	learn: 0.7711545	total: 1m 1s	remaining: 1m 28s
329:	learn: 0.7712965	total: 1m 1s	remaining: 1m 27s
330:	learn: 0.7714345	total: 1m 1s	remaining: 1m 27s
331:	learn: 0.7715644	total: 1m 2s	remaining: 1m 27s
332:	learn: 0.7716522	total: 1m 2s	remaining: 1m 27s
33

470:	learn: 0.7901401	total: 1m 24s	remaining: 59s
471:	learn: 0.7902669	total: 1m 24s	remaining: 58.8s
472:	learn: 0.7904110	total: 1m 24s	remaining: 58.6s
473:	learn: 0.7905315	total: 1m 24s	remaining: 58.4s
474:	learn: 0.7907531	total: 1m 25s	remaining: 58.2s
475:	learn: 0.7908482	total: 1m 25s	remaining: 58s
476:	learn: 0.7909986	total: 1m 25s	remaining: 57.8s
477:	learn: 0.7911705	total: 1m 25s	remaining: 57.7s
478:	learn: 0.7912656	total: 1m 25s	remaining: 57.6s
479:	learn: 0.7913697	total: 1m 26s	remaining: 57.4s
480:	learn: 0.7914414	total: 1m 26s	remaining: 57.2s
481:	learn: 0.7915502	total: 1m 26s	remaining: 57s
482:	learn: 0.7916950	total: 1m 26s	remaining: 56.8s
483:	learn: 0.7918750	total: 1m 26s	remaining: 56.6s
484:	learn: 0.7920553	total: 1m 26s	remaining: 56.4s
485:	learn: 0.7921709	total: 1m 27s	remaining: 56.3s
486:	learn: 0.7923051	total: 1m 27s	remaining: 56.1s
487:	learn: 0.7924624	total: 1m 27s	remaining: 55.9s
488:	learn: 0.7926188	total: 1m 27s	remaining: 55.7s

626:	learn: 0.8086092	total: 1m 51s	remaining: 30.8s
627:	learn: 0.8087439	total: 1m 51s	remaining: 30.6s
628:	learn: 0.8089046	total: 1m 51s	remaining: 30.4s
629:	learn: 0.8090094	total: 1m 52s	remaining: 30.3s
630:	learn: 0.8091541	total: 1m 52s	remaining: 30.1s
631:	learn: 0.8092303	total: 1m 52s	remaining: 30s
632:	learn: 0.8093641	total: 1m 53s	remaining: 29.9s
633:	learn: 0.8095425	total: 1m 53s	remaining: 29.7s
634:	learn: 0.8096174	total: 1m 53s	remaining: 29.6s
635:	learn: 0.8096810	total: 1m 53s	remaining: 29.4s
636:	learn: 0.8097458	total: 1m 54s	remaining: 29.2s
637:	learn: 0.8098784	total: 1m 54s	remaining: 29s
638:	learn: 0.8099854	total: 1m 54s	remaining: 28.9s
639:	learn: 0.8101049	total: 1m 54s	remaining: 28.7s
640:	learn: 0.8101831	total: 1m 54s	remaining: 28.5s
641:	learn: 0.8102533	total: 1m 55s	remaining: 28.3s
642:	learn: 0.8103736	total: 1m 55s	remaining: 28.1s
643:	learn: 0.8104676	total: 1m 55s	remaining: 28s
644:	learn: 0.8105776	total: 1m 55s	remaining: 27.8s

784:	learn: 0.8245858	total: 2m 23s	remaining: 2.74s
785:	learn: 0.8246866	total: 2m 23s	remaining: 2.56s
786:	learn: 0.8247698	total: 2m 23s	remaining: 2.38s
787:	learn: 0.8248334	total: 2m 24s	remaining: 2.19s
788:	learn: 0.8249086	total: 2m 24s	remaining: 2.01s
789:	learn: 0.8250124	total: 2m 24s	remaining: 1.83s
790:	learn: 0.8251503	total: 2m 24s	remaining: 1.64s
791:	learn: 0.8252222	total: 2m 24s	remaining: 1.46s
792:	learn: 0.8253365	total: 2m 24s	remaining: 1.28s
793:	learn: 0.8253860	total: 2m 25s	remaining: 1.09s
794:	learn: 0.8254686	total: 2m 25s	remaining: 913ms
795:	learn: 0.8255485	total: 2m 25s	remaining: 730ms
796:	learn: 0.8256546	total: 2m 25s	remaining: 547ms
797:	learn: 0.8257056	total: 2m 25s	remaining: 365ms
798:	learn: 0.8258321	total: 2m 25s	remaining: 182ms
799:	learn: 0.8259416	total: 2m 25s	remaining: 0us


In [235]:
ctb_model.predict_proba(test_X)[:,1].size

4166

In [236]:
dropped_test['0'] = ctb_model.predict_proba(test_X)[:,1]
dropped_test['0'].to_csv('submissionCBmain.csv',header = True, index_label = True)
dropped_test['0']

0       0.127997
1       0.595952
2       0.188427
3       0.338136
4       0.501206
5       0.300054
6       0.097902
7       0.146040
8       0.119063
9       0.295177
10      0.138115
11      0.573523
12      0.070019
13      0.128807
14      0.648467
15      0.169574
16      0.054488
17      0.204460
18      0.333342
19      0.113048
20      0.049054
21      0.252240
22      0.156741
23      0.053239
24      0.621296
25      0.312374
26      0.365252
27      0.103548
28      0.087859
29      0.164393
          ...   
4136    0.080775
4137    0.054327
4138    0.073669
4139    0.109595
4140    0.116617
4141    0.232858
4142    0.058112
4143    0.137284
4144    0.062445
4145    0.127642
4146    0.070277
4147    0.087465
4148    0.084094
4149    0.188409
4150    0.122239
4151    0.083666
4152    0.110933
4153    0.349383
4154    0.289059
4155    0.137924
4156    0.586188
4157    0.184190
4158    0.185031
4159    0.284549
4160    0.059453
4161    0.082153
4162    0.075336
4163    0.0813

In [333]:
prediction_lgbm = pd.Series( index = dropped_train.index)
prediction_cat = pd.Series( index = dropped_train.index )
for train, val in skf.split(X,y):
    temp_lgb_train = lgb.Dataset(X.loc[train], label=y.loc[train], free_raw_data=False, categorical_feature = categ_columns)
    temp_lgbm = lgb.train(parameters, temp_lgb_train, 730)
    temp_ctb_model = ctb.CatBoostClassifier(iterations=800, learning_rate=0.03, depth=6, eval_metric='AUC', 
                                       random_seed=42, rsm=0.8, l2_leaf_reg=2, od_wait=50, od_type='Iter')
    temp_ctb_model.fit(X.loc[train], y.loc[train], cat_features = np.where(X.dtypes != np.float)[0],verbose=True)
    prediction_lgbm[val] = temp_lgbm.predict(X.loc[val])
    prediction_cat[val] = temp_ctb_model.predict_proba(X.loc[val])[:,1]

/home/anton/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/anton/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


0:	learn: 0.6333750	total: 118ms	remaining: 1m 34s
1:	learn: 0.6660129	total: 226ms	remaining: 1m 30s
2:	learn: 0.6662254	total: 362ms	remaining: 1m 36s
3:	learn: 0.6744452	total: 466ms	remaining: 1m 32s
4:	learn: 0.6732678	total: 582ms	remaining: 1m 32s
5:	learn: 0.6726698	total: 682ms	remaining: 1m 30s
6:	learn: 0.6784979	total: 798ms	remaining: 1m 30s
7:	learn: 0.6792081	total: 931ms	remaining: 1m 32s
8:	learn: 0.6872419	total: 1.08s	remaining: 1m 34s
9:	learn: 0.6882150	total: 1.22s	remaining: 1m 36s
10:	learn: 0.6873034	total: 1.4s	remaining: 1m 40s
11:	learn: 0.6869677	total: 1.51s	remaining: 1m 39s
12:	learn: 0.6885363	total: 1.63s	remaining: 1m 38s
13:	learn: 0.6874810	total: 1.74s	remaining: 1m 37s
14:	learn: 0.6890522	total: 1.86s	remaining: 1m 37s
15:	learn: 0.6936755	total: 2.01s	remaining: 1m 38s
16:	learn: 0.6936298	total: 2.18s	remaining: 1m 40s
17:	learn: 0.6961929	total: 2.28s	remaining: 1m 39s
18:	learn: 0.6956560	total: 2.4s	remaining: 1m 38s
19:	learn: 0.6955255	tot

158:	learn: 0.7506144	total: 19.6s	remaining: 1m 19s
159:	learn: 0.7507223	total: 19.7s	remaining: 1m 18s
160:	learn: 0.7508184	total: 19.8s	remaining: 1m 18s
161:	learn: 0.7510524	total: 20s	remaining: 1m 18s
162:	learn: 0.7513054	total: 20.1s	remaining: 1m 18s
163:	learn: 0.7513548	total: 20.2s	remaining: 1m 18s
164:	learn: 0.7515454	total: 20.3s	remaining: 1m 18s
165:	learn: 0.7518831	total: 20.4s	remaining: 1m 18s
166:	learn: 0.7519942	total: 20.5s	remaining: 1m 17s
167:	learn: 0.7521989	total: 20.7s	remaining: 1m 17s
168:	learn: 0.7524119	total: 20.8s	remaining: 1m 17s
169:	learn: 0.7525250	total: 21s	remaining: 1m 17s
170:	learn: 0.7526586	total: 21.1s	remaining: 1m 17s
171:	learn: 0.7527516	total: 21.2s	remaining: 1m 17s
172:	learn: 0.7531100	total: 21.4s	remaining: 1m 17s
173:	learn: 0.7532648	total: 21.5s	remaining: 1m 17s
174:	learn: 0.7535371	total: 21.6s	remaining: 1m 17s
175:	learn: 0.7536666	total: 21.7s	remaining: 1m 16s
176:	learn: 0.7538441	total: 21.8s	remaining: 1m 1

315:	learn: 0.7775253	total: 39.9s	remaining: 1m 1s
316:	learn: 0.7777623	total: 40.1s	remaining: 1m 1s
317:	learn: 0.7778767	total: 40.2s	remaining: 1m
318:	learn: 0.7779697	total: 40.3s	remaining: 1m
319:	learn: 0.7781166	total: 40.4s	remaining: 1m
320:	learn: 0.7781548	total: 40.5s	remaining: 1m
321:	learn: 0.7783079	total: 40.6s	remaining: 1m
322:	learn: 0.7784884	total: 40.7s	remaining: 1m
323:	learn: 0.7785682	total: 40.9s	remaining: 1m
324:	learn: 0.7787653	total: 41s	remaining: 59.9s
325:	learn: 0.7788905	total: 41.1s	remaining: 59.8s
326:	learn: 0.7790935	total: 41.2s	remaining: 59.6s
327:	learn: 0.7792000	total: 41.3s	remaining: 59.5s
328:	learn: 0.7792950	total: 41.5s	remaining: 59.4s
329:	learn: 0.7795429	total: 41.6s	remaining: 59.2s
330:	learn: 0.7796601	total: 41.7s	remaining: 59.1s
331:	learn: 0.7797880	total: 41.8s	remaining: 59s
332:	learn: 0.7799078	total: 41.9s	remaining: 58.8s
333:	learn: 0.7800491	total: 42s	remaining: 58.7s
334:	learn: 0.7802262	total: 42.2s	rema

476:	learn: 0.8019051	total: 59.1s	remaining: 40s
477:	learn: 0.8020836	total: 59.2s	remaining: 39.9s
478:	learn: 0.8022876	total: 59.3s	remaining: 39.7s
479:	learn: 0.8023042	total: 59.4s	remaining: 39.6s
480:	learn: 0.8025148	total: 59.5s	remaining: 39.5s
481:	learn: 0.8026861	total: 59.6s	remaining: 39.3s
482:	learn: 0.8028572	total: 59.7s	remaining: 39.2s
483:	learn: 0.8029595	total: 59.8s	remaining: 39s
484:	learn: 0.8031145	total: 59.9s	remaining: 38.9s
485:	learn: 0.8032179	total: 60s	remaining: 38.7s
486:	learn: 0.8033958	total: 1m	remaining: 38.6s
487:	learn: 0.8036531	total: 1m	remaining: 38.5s
488:	learn: 0.8037359	total: 1m	remaining: 38.4s
489:	learn: 0.8038209	total: 1m	remaining: 38.3s
490:	learn: 0.8039811	total: 1m	remaining: 38.2s
491:	learn: 0.8040883	total: 1m	remaining: 38s
492:	learn: 0.8042072	total: 1m	remaining: 37.9s
493:	learn: 0.8043186	total: 1m 1s	remaining: 37.8s
494:	learn: 0.8045151	total: 1m 1s	remaining: 37.7s
495:	learn: 0.8046321	total: 1m 1s	remain

...

110:	learn: 0.7333044	total: 17.6s	remaining: 1m 49s
111:	learn: 0.7334419	total: 17.7s	remaining: 1m 48s
112:	learn: 0.7337232	total: 17.8s	remaining: 1m 48s
113:	learn: 0.7340502	total: 17.9s	remaining: 1m 47s
114:	learn: 0.7343120	total: 18s	remaining: 1m 47s
115:	learn: 0.7350414	total: 18.1s	remaining: 1m 46s
116:	learn: 0.7353099	total: 18.2s	remaining: 1m 46s
117:	learn: 0.7356072	total: 18.3s	remaining: 1m 46s
118:	learn: 0.7357668	total: 18.4s	remaining: 1m 45s
119:	learn: 0.7358972	total: 18.5s	remaining: 1m 44s
120:	learn: 0.7361506	total: 18.6s	remaining: 1m 44s
121:	learn: 0.7363640	total: 18.7s	remaining: 1m 44s
122:	learn: 0.7366397	total: 18.8s	remaining: 1m 43s
123:	learn: 0.7369136	total: 18.9s	remaining: 1m 43s
124:	learn: 0.7370431	total: 19s	remaining: 1m 42s
125:	learn: 0.7374835	total: 19.1s	remaining: 1m 42s
126:	learn: 0.7377613	total: 19.2s	remaining: 1m 41s
127:	learn: 0.7379509	total: 19.3s	remaining: 1m 41s
128:	learn: 0.7383538	total: 19.5s	remaining: 1m 4

267:	learn: 0.7649332	total: 35.2s	remaining: 1m 9s
268:	learn: 0.7651795	total: 35.3s	remaining: 1m 9s
269:	learn: 0.7653103	total: 35.5s	remaining: 1m 9s
270:	learn: 0.7656624	total: 35.6s	remaining: 1m 9s
271:	learn: 0.7657947	total: 35.7s	remaining: 1m 9s
272:	learn: 0.7658901	total: 35.8s	remaining: 1m 9s
273:	learn: 0.7660132	total: 35.9s	remaining: 1m 8s
274:	learn: 0.7661245	total: 36.1s	remaining: 1m 8s
275:	learn: 0.7662739	total: 36.2s	remaining: 1m 8s
276:	learn: 0.7664774	total: 36.3s	remaining: 1m 8s
277:	learn: 0.7666290	total: 36.5s	remaining: 1m 8s
278:	learn: 0.7667834	total: 36.6s	remaining: 1m 8s
279:	learn: 0.7668791	total: 36.7s	remaining: 1m 8s
280:	learn: 0.7670984	total: 36.9s	remaining: 1m 8s
281:	learn: 0.7671870	total: 37s	remaining: 1m 7s
282:	learn: 0.7673216	total: 37.1s	remaining: 1m 7s
283:	learn: 0.7674580	total: 37.2s	remaining: 1m 7s
284:	learn: 0.7675919	total: 37.3s	remaining: 1m 7s
285:	learn: 0.7677106	total: 37.5s	remaining: 1m 7s
286:	learn: 0.

427:	learn: 0.7914076	total: 54.5s	remaining: 47.4s
428:	learn: 0.7916437	total: 54.6s	remaining: 47.2s
429:	learn: 0.7917702	total: 54.8s	remaining: 47.1s
430:	learn: 0.7918465	total: 54.9s	remaining: 47s
431:	learn: 0.7919993	total: 55s	remaining: 46.9s
432:	learn: 0.7921099	total: 55.1s	remaining: 46.7s
433:	learn: 0.7922710	total: 55.2s	remaining: 46.6s
434:	learn: 0.7925126	total: 55.3s	remaining: 46.4s
435:	learn: 0.7927122	total: 55.5s	remaining: 46.3s
436:	learn: 0.7928641	total: 55.6s	remaining: 46.2s
437:	learn: 0.7930903	total: 55.7s	remaining: 46s
438:	learn: 0.7932121	total: 55.8s	remaining: 45.9s
439:	learn: 0.7934234	total: 55.9s	remaining: 45.7s
440:	learn: 0.7935233	total: 56s	remaining: 45.6s
441:	learn: 0.7937064	total: 56.1s	remaining: 45.4s
442:	learn: 0.7939894	total: 56.2s	remaining: 45.3s
443:	learn: 0.7941127	total: 56.3s	remaining: 45.2s
444:	learn: 0.7942015	total: 56.4s	remaining: 45s
445:	learn: 0.7943162	total: 56.5s	remaining: 44.9s
446:	learn: 0.7944825	

586:	learn: 0.8140350	total: 1m 20s	remaining: 29.3s
587:	learn: 0.8142193	total: 1m 20s	remaining: 29.2s
588:	learn: 0.8143491	total: 1m 21s	remaining: 29s
589:	learn: 0.8144926	total: 1m 21s	remaining: 28.9s
590:	learn: 0.8146727	total: 1m 21s	remaining: 28.7s
591:	learn: 0.8147698	total: 1m 21s	remaining: 28.6s
592:	learn: 0.8149225	total: 1m 21s	remaining: 28.4s
593:	learn: 0.8151398	total: 1m 21s	remaining: 28.3s
594:	learn: 0.8152514	total: 1m 21s	remaining: 28.1s
595:	learn: 0.8153926	total: 1m 21s	remaining: 28s
596:	learn: 0.8155185	total: 1m 21s	remaining: 27.8s
597:	learn: 0.8156354	total: 1m 21s	remaining: 27.7s
598:	learn: 0.8158060	total: 1m 21s	remaining: 27.5s
599:	learn: 0.8158860	total: 1m 22s	remaining: 27.4s
600:	learn: 0.8160492	total: 1m 22s	remaining: 27.2s
601:	learn: 0.8161819	total: 1m 22s	remaining: 27.1s
602:	learn: 0.8162871	total: 1m 22s	remaining: 26.9s
603:	learn: 0.8163334	total: 1m 22s	remaining: 26.8s
604:	learn: 0.8165207	total: 1m 22s	remaining: 26.

743:	learn: 0.8325579	total: 1m 41s	remaining: 7.62s
744:	learn: 0.8327171	total: 1m 41s	remaining: 7.48s
745:	learn: 0.8328546	total: 1m 41s	remaining: 7.35s
746:	learn: 0.8329108	total: 1m 41s	remaining: 7.21s
747:	learn: 0.8330283	total: 1m 41s	remaining: 7.07s
748:	learn: 0.8331346	total: 1m 41s	remaining: 6.93s
749:	learn: 0.8332514	total: 1m 41s	remaining: 6.79s
750:	learn: 0.8334170	total: 1m 42s	remaining: 6.66s
751:	learn: 0.8336066	total: 1m 42s	remaining: 6.52s
752:	learn: 0.8336836	total: 1m 42s	remaining: 6.38s
753:	learn: 0.8338196	total: 1m 42s	remaining: 6.24s
754:	learn: 0.8339559	total: 1m 42s	remaining: 6.11s
755:	learn: 0.8340580	total: 1m 42s	remaining: 5.97s
756:	learn: 0.8342141	total: 1m 42s	remaining: 5.83s
757:	learn: 0.8342774	total: 1m 42s	remaining: 5.7s
758:	learn: 0.8343553	total: 1m 42s	remaining: 5.56s
759:	learn: 0.8344057	total: 1m 42s	remaining: 5.42s
760:	learn: 0.8345387	total: 1m 43s	remaining: 5.28s
761:	learn: 0.8346440	total: 1m 43s	remaining: 

In [334]:
roc_auc_score(y, np.sqrt(prediction_lgbm*prediction_cat))

0.7452001497437402

In [335]:
roc_auc_score(y, (0.5*prediction_lgbm+0.5*prediction_cat))

0.7451562617904408

In [336]:
roc_auc_score(y, (0.7*prediction_lgbm+0.3*prediction_cat))

0.7453866476067555

In [337]:
roc_auc_score(y, (0.3*prediction_lgbm+0.7*prediction_cat))

0.7442614351870606

In [342]:
dropped_test['0'] = (0.7*lgbm.predict(test_X) +0.3*ctb_model.predict_proba(test_X)[:,1])
dropped_test['0'].to_csv('submissionEns.csv',header = True, index_label = True)
dropped_test['0']

0       0.090354
1       0.362467
2       0.276801
3       0.224088
4       0.507119
5       0.259894
6       0.061547
7       0.089206
8       0.062753
9       0.243995
10      0.081278
11      0.291771
12      0.023404
13      0.067593
14      0.493130
15      0.164073
16      0.048084
17      0.172175
18      0.210746
19      0.059096
20      0.031200
21      0.157957
22      0.117107
23      0.027978
24      0.401891
25      0.193057
26      0.296597
27      0.077560
28      0.040864
29      0.122519
          ...   
4136    0.066486
4137    0.031824
4138    0.038924
4139    0.078221
4140    0.068261
4141    0.120857
4142    0.043917
4143    0.069013
4144    0.056591
4145    0.110812
4146    0.030530
4147    0.094797
4148    0.062825
4149    0.140794
4150    0.088679
4151    0.069534
4152    0.087504
4153    0.221414
4154    0.152193
4155    0.145829
4156    0.376078
4157    0.080382
4158    0.112597
4159    0.209940
4160    0.028292
4161    0.054213
4162    0.059987
4163    0.0596

In [350]:
X['pred1'] = prediction_lgbm
X

,1,2,3,4,6,7,8,9,10,12,...,337,338,339,340,341,342,343,344,345,pred1
0,1,0,0,0,0,0.090909,0,0,1,1,...,0,1,0,0,0.222222,1,1,1,1,0.511495
1,1,0,0,1,0,0.090909,0,0,1,1,...,0,1,0,0,0.111111,1,1,1,0,0.191582
2,1,0,0,1,0,0.090909,0,0,1,1,...,0,1,0,0,0.444444,1,1,1,1,0.259226
3,1,0,0,1,0,0.136364,0,0,1,1,...,0,0,1,0,0.222222,1,1,1,0,0.207168
4,1,0,0,1,0,0.136364,0,0,1,1,...,0,0,1,0,0.111111,1,1,1,1,0.673893
5,1,0,0,0,0,0.090909,0,0,0,1,...,0,0,0,0,0.000000,1,1,1,0,0.089646
6,1,0,0,0,0,0.090909,0,0,1,1,...,1,0,0,0,0.000000,1,1,1,0,0.154342
7,1,0,0,1,0,0.090909,0,0,1,1,...,0,1,0,0,0.000000,1,1,1,0,0.086208
8,1,0,0,0,0,0.090909,0,0,1,1,...,1,0,0,0,0.000000,1,1,1,1,0.129071
9,1,0,0,0,0,0.136364,0,0,1,1,...,1,0,0,0,0.000000,1,1,1,0,0.055923


In [351]:
test_X['pred1'] = lgbm.predict(test_X)

In [354]:
lgb_train = lgb.Dataset(X, label=y, free_raw_data=False, categorical_feature=categ_columns)

In [355]:
parameters = {
    #
    'objective': 'binary',
    'learning_rate': 0.01,
    'num_threads': -1,
    "metric": 'auc',
    'num_leaves': 63,
    
    #regularization
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'subsample_freq': 1,
    'min_data_in_leaf': 15,
    'cat_smooth': 10,
    'min_data_per_group': 100
}
result = lgb.cv(parameters, lgb_train, n_rounds, folds=skf.split(X, y), early_stopping_rounds=400, verbose_eval=2)

/home/anton/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[2]	cv_agg's auc: 0.725454 + 0.0101522
[4]	cv_agg's auc: 0.736799 + 0.0112623
[6]	cv_agg's auc: 0.737468 + 0.0118866
[8]	cv_agg's auc: 0.738248 + 0.0113585
[10]	cv_agg's auc: 0.73976 + 0.0115729
[12]	cv_agg's auc: 0.740783 + 0.0114853
[14]	cv_agg's auc: 0.741352 + 0.0115208
[16]	cv_agg's auc: 0.741732 + 0.0116186
[18]	cv_agg's auc: 0.741993 + 0.011581
[20]	cv_agg's auc: 0.74193 + 0.0115002
[22]	cv_agg's auc: 0.741815 + 0.0112966
[24]	cv_agg's auc: 0.741447 + 0.010998
[26]	cv_agg's auc: 0.741682 + 0.01143
[28]	cv_agg's auc: 0.74138 + 0.0108402
[30]	cv_agg's auc: 0.741722 + 0.0109526
[32]	cv_agg's auc: 0.741387 + 0.0110257
[34]	cv_agg's auc: 0.741632 + 0.0110558
[36]	cv_agg's auc: 0.741769 + 0.0112026
[38]	cv_agg's auc: 0.741939 + 0.0113051
[40]	cv_agg's auc: 0.742154 + 0.0114342
[42]	cv_agg's auc: 0.742034 + 0.0112099
[44]	cv_agg's auc: 0.742131 + 0.0111736
[46]	cv_agg's auc: 0.742062 + 0.0111951
[48]	cv_agg's auc: 0.742158 + 0.0112494
[50]	cv_agg's auc: 0.742196 + 0.0112969
[52]	cv_agg

[408]	cv_agg's auc: 0.740076 + 0.0108959
[410]	cv_agg's auc: 0.740064 + 0.0109144
[412]	cv_agg's auc: 0.740036 + 0.0109129
[414]	cv_agg's auc: 0.740034 + 0.0109882
[416]	cv_agg's auc: 0.739986 + 0.0109789
[418]	cv_agg's auc: 0.739996 + 0.0109451
[420]	cv_agg's auc: 0.73997 + 0.0109673
[422]	cv_agg's auc: 0.739931 + 0.0109591
[424]	cv_agg's auc: 0.739878 + 0.0108862
[426]	cv_agg's auc: 0.739885 + 0.0109346
[428]	cv_agg's auc: 0.739834 + 0.010955
[430]	cv_agg's auc: 0.739819 + 0.0108922
[432]	cv_agg's auc: 0.739798 + 0.010944
[434]	cv_agg's auc: 0.739806 + 0.0109517
[436]	cv_agg's auc: 0.739831 + 0.0109394
[438]	cv_agg's auc: 0.739843 + 0.0109217
[440]	cv_agg's auc: 0.739845 + 0.0109205
[442]	cv_agg's auc: 0.739851 + 0.0109538
[444]	cv_agg's auc: 0.739848 + 0.010997
[446]	cv_agg's auc: 0.739803 + 0.0109817
[448]	cv_agg's auc: 0.739809 + 0.0109339
[450]	cv_agg's auc: 0.739841 + 0.0109529
[452]	cv_agg's auc: 0.739846 + 0.0108984
[454]	cv_agg's auc: 0.739844 + 0.0109099
[456]	cv_agg's auc: 

In [356]:
lgbm = lgb.train(parameters, lgb_train, 200)

/home/anton/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


In [357]:
dropped_test['0'] = lgbm.predict(test_X)
dropped_test['0'].to_csv('submissionLGBST.csv',header = True, index_label = True)

In [352]:
ctb_model = ctb.CatBoostClassifier(iterations=800, learning_rate=0.03, depth=6, eval_metric='AUC', 
                                       random_seed=42, rsm=0.8, l2_leaf_reg=2, od_wait=50, od_type='Iter')
ctb_model.fit(X, y, np.where(X.dtypes != np.float)[0], verbose=True)

0:	learn: 0.7000644	total: 140ms	remaining: 1m 52s
1:	learn: 0.7016370	total: 286ms	remaining: 1m 54s
2:	learn: 0.7076851	total: 441ms	remaining: 1m 57s
3:	learn: 0.7105188	total: 602ms	remaining: 1m 59s
4:	learn: 0.7125692	total: 808ms	remaining: 2m 8s
5:	learn: 0.7148664	total: 937ms	remaining: 2m 4s
6:	learn: 0.7163895	total: 1.09s	remaining: 2m 3s
7:	learn: 0.7166284	total: 1.25s	remaining: 2m 3s
8:	learn: 0.7385087	total: 1.4s	remaining: 2m 3s
9:	learn: 0.7408977	total: 1.54s	remaining: 2m 1s
10:	learn: 0.7427736	total: 1.69s	remaining: 2m 1s
11:	learn: 0.7422973	total: 1.85s	remaining: 2m 1s
12:	learn: 0.7423383	total: 2s	remaining: 2m 1s
13:	learn: 0.7427883	total: 2.14s	remaining: 2m
14:	learn: 0.7438733	total: 2.29s	remaining: 2m
15:	learn: 0.7439334	total: 2.43s	remaining: 1m 59s
16:	learn: 0.7442757	total: 2.58s	remaining: 1m 58s
17:	learn: 0.7454060	total: 2.72s	remaining: 1m 58s
18:	learn: 0.7454744	total: 2.85s	remaining: 1m 57s
19:	learn: 0.7454764	total: 2.94s	remaining

158:	learn: 0.7561684	total: 31.4s	remaining: 2m 6s
159:	learn: 0.7562851	total: 31.6s	remaining: 2m 6s
160:	learn: 0.7563760	total: 31.7s	remaining: 2m 5s
161:	learn: 0.7564010	total: 31.8s	remaining: 2m 5s
162:	learn: 0.7564834	total: 32s	remaining: 2m 5s
163:	learn: 0.7566252	total: 32.1s	remaining: 2m 4s
164:	learn: 0.7566762	total: 32.3s	remaining: 2m 4s
165:	learn: 0.7567148	total: 32.4s	remaining: 2m 3s
166:	learn: 0.7568471	total: 32.6s	remaining: 2m 3s
167:	learn: 0.7569650	total: 32.7s	remaining: 2m 3s
168:	learn: 0.7570199	total: 32.8s	remaining: 2m 2s
169:	learn: 0.7570591	total: 33s	remaining: 2m 2s
170:	learn: 0.7571917	total: 33.1s	remaining: 2m 1s
171:	learn: 0.7572426	total: 33.3s	remaining: 2m 1s
172:	learn: 0.7573572	total: 33.4s	remaining: 2m 1s
173:	learn: 0.7574421	total: 33.5s	remaining: 2m
174:	learn: 0.7575137	total: 33.7s	remaining: 2m
175:	learn: 0.7575493	total: 33.8s	remaining: 1m 59s
176:	learn: 0.7576460	total: 34s	remaining: 1m 59s
177:	learn: 0.7576961	

314:	learn: 0.7680513	total: 59.2s	remaining: 1m 31s
315:	learn: 0.7681246	total: 59.6s	remaining: 1m 31s
316:	learn: 0.7681978	total: 59.7s	remaining: 1m 30s
317:	learn: 0.7682743	total: 59.9s	remaining: 1m 30s
318:	learn: 0.7683289	total: 1m	remaining: 1m 30s
319:	learn: 0.7684416	total: 1m	remaining: 1m 30s
320:	learn: 0.7685804	total: 1m	remaining: 1m 30s
321:	learn: 0.7686539	total: 1m	remaining: 1m 29s
322:	learn: 0.7687080	total: 1m	remaining: 1m 29s
323:	learn: 0.7688272	total: 1m	remaining: 1m 29s
324:	learn: 0.7689228	total: 1m 1s	remaining: 1m 29s
325:	learn: 0.7689596	total: 1m 1s	remaining: 1m 29s
326:	learn: 0.7690399	total: 1m 1s	remaining: 1m 29s
327:	learn: 0.7690740	total: 1m 1s	remaining: 1m 29s
328:	learn: 0.7691412	total: 1m 2s	remaining: 1m 29s
329:	learn: 0.7692697	total: 1m 2s	remaining: 1m 28s
330:	learn: 0.7693813	total: 1m 2s	remaining: 1m 28s
331:	learn: 0.7694717	total: 1m 2s	remaining: 1m 28s
332:	learn: 0.7695589	total: 1m 2s	remaining: 1m 28s
333:	learn:

468:	learn: 0.7833354	total: 1m 31s	remaining: 1m 4s
469:	learn: 0.7834275	total: 1m 31s	remaining: 1m 4s
470:	learn: 0.7834888	total: 1m 32s	remaining: 1m 4s
471:	learn: 0.7836140	total: 1m 32s	remaining: 1m 4s
472:	learn: 0.7837246	total: 1m 32s	remaining: 1m 3s
473:	learn: 0.7838943	total: 1m 32s	remaining: 1m 3s
474:	learn: 0.7840220	total: 1m 32s	remaining: 1m 3s
475:	learn: 0.7841866	total: 1m 32s	remaining: 1m 3s
476:	learn: 0.7842973	total: 1m 32s	remaining: 1m 2s
477:	learn: 0.7843608	total: 1m 33s	remaining: 1m 2s
478:	learn: 0.7844550	total: 1m 33s	remaining: 1m 2s
479:	learn: 0.7846230	total: 1m 33s	remaining: 1m 2s
480:	learn: 0.7847081	total: 1m 33s	remaining: 1m 2s
481:	learn: 0.7847965	total: 1m 33s	remaining: 1m 1s
482:	learn: 0.7848773	total: 1m 33s	remaining: 1m 1s
483:	learn: 0.7849618	total: 1m 33s	remaining: 1m 1s
484:	learn: 0.7850803	total: 1m 34s	remaining: 1m 1s
485:	learn: 0.7851819	total: 1m 34s	remaining: 1m
486:	learn: 0.7852965	total: 1m 34s	remaining: 1m

626:	learn: 0.7989220	total: 2m 3s	remaining: 34s
627:	learn: 0.7989894	total: 2m 3s	remaining: 33.8s
628:	learn: 0.7990710	total: 2m 3s	remaining: 33.6s
629:	learn: 0.7991156	total: 2m 3s	remaining: 33.4s
630:	learn: 0.7992474	total: 2m 3s	remaining: 33.2s
631:	learn: 0.7993450	total: 2m 4s	remaining: 33s
632:	learn: 0.7994205	total: 2m 4s	remaining: 32.8s
633:	learn: 0.7994834	total: 2m 4s	remaining: 32.6s
634:	learn: 0.7995555	total: 2m 4s	remaining: 32.4s
635:	learn: 0.7997072	total: 2m 4s	remaining: 32.2s
636:	learn: 0.7998003	total: 2m 4s	remaining: 31.9s
637:	learn: 0.7999185	total: 2m 5s	remaining: 31.7s
638:	learn: 0.8001090	total: 2m 5s	remaining: 31.5s
639:	learn: 0.8001773	total: 2m 5s	remaining: 31.3s
640:	learn: 0.8002240	total: 2m 5s	remaining: 31.1s
641:	learn: 0.8003182	total: 2m 5s	remaining: 30.9s
642:	learn: 0.8003932	total: 2m 5s	remaining: 30.7s
643:	learn: 0.8004738	total: 2m 6s	remaining: 30.5s
644:	learn: 0.8006214	total: 2m 6s	remaining: 30.3s
645:	learn: 0.80

784:	learn: 0.8126978	total: 2m 40s	remaining: 3.06s
785:	learn: 0.8127663	total: 2m 40s	remaining: 2.86s
786:	learn: 0.8128494	total: 2m 40s	remaining: 2.65s
787:	learn: 0.8129133	total: 2m 40s	remaining: 2.45s
788:	learn: 0.8129750	total: 2m 40s	remaining: 2.24s
789:	learn: 0.8130267	total: 2m 41s	remaining: 2.04s
790:	learn: 0.8131609	total: 2m 41s	remaining: 1.83s
791:	learn: 0.8133070	total: 2m 41s	remaining: 1.63s
792:	learn: 0.8133583	total: 2m 41s	remaining: 1.43s
793:	learn: 0.8134629	total: 2m 41s	remaining: 1.22s
794:	learn: 0.8134938	total: 2m 41s	remaining: 1.02s
795:	learn: 0.8135479	total: 2m 42s	remaining: 814ms
796:	learn: 0.8135891	total: 2m 42s	remaining: 611ms
797:	learn: 0.8137105	total: 2m 42s	remaining: 407ms
798:	learn: 0.8138510	total: 2m 42s	remaining: 203ms
799:	learn: 0.8139781	total: 2m 42s	remaining: 0us


In [353]:
dropped_test['0'] = (ctb_model.predict_proba(test_X)[:,1])
dropped_test['0'].to_csv('submissionST.csv',header = True, index_label = True)
dropped_test['0']

0       0.124818
1       0.549705
2       0.433956
3       0.310356
4       0.786224
5       0.402943
6       0.078616
7       0.129710
8       0.097170
9       0.328071
10      0.071158
11      0.452148
12      0.023417
13      0.082472
14      0.709996
15      0.226394
16      0.066091
17      0.295015
18      0.291525
19      0.083076
20      0.037346
21      0.190363
22      0.151514
23      0.037098
24      0.587612
25      0.343085
26      0.467112
27      0.094550
28      0.061947
29      0.185752
          ...   
4136    0.090500
4137    0.036808
4138    0.049491
4139    0.106500
4140    0.098113
4141    0.178586
4142    0.047309
4143    0.099233
4144    0.080051
4145    0.155309
4146    0.040077
4147    0.169299
4148    0.099543
4149    0.246903
4150    0.157255
4151    0.110937
4152    0.114726
4153    0.343856
4154    0.233946
4155    0.213057
4156    0.609890
4157    0.127313
4158    0.149048
4159    0.349339
4160    0.047602
4161    0.075387
4162    0.075793
4163    0.0834